In [10]:
import time
import random
import pandas as pd
import mysql.connector
import psycopg2
from sqlalchemy import create_engine
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Database Connection Details
MYSQL_DB = {
    "host": "localhost",
    "user": "root",
    "password": "password",
    "database": "scraping",
}

POSTGRESQL_DB = {
    "host": "localhost",
    "user": "postgres",
    "password": "password",
    "database": "scraping",
}

# Setup Selenium WebDriver with Headless Mode
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36")

service = Service("chromedriver.exe")  # Ensure correct chromedriver path

# Load the CSV file with Twitter links
df = pd.read_csv("twitter_links.csv", header=None)
df.columns = ["twitter_link"]
df["username"] = df["twitter_link"].str.extract(r"twitter\.com\/@?([^\/\s]+)")

# List to store profile data
profile_data = []

# Function to scrape Twitter profiles
def scrape_twitter_profile(username, driver):
    if pd.isna(username):  # If there's no username, mark as "No Profile Present"
        return ["No Profile Present", "N/A", "N/A", "N/A", "N/A", "N/A"]
    
    twitter_url = f"https://twitter.com/{username}"
    driver.get(twitter_url)
    
    try:
        if username == df["username"].iloc[0]:  # First row
            WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[data-testid="UserDescription"], span[data-testid="UserLocation"]')))
        else:
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[data-testid="UserDescription"], span[data-testid="UserLocation"]'))
        )
    except Exception as e:
        print(f"Timeout or CAPTCHA issue with {username}: {e}")
        return [username, "N/A", "N/A", "N/A", "N/A", "N/A"]
    
    soup = BeautifulSoup(driver.page_source, "html.parser")

    def extract_data(selector, attribute="text"):
        try:
            element = soup.select_one(selector)
            return element.text.strip() if attribute == "text" else element[attribute]
        except (AttributeError, TypeError):
            return "N/A"

    bio = extract_data('div[data-testid="UserDescription"]')
    followers = extract_data(f'a[href="/{username}/followers"] span')
    following = extract_data(f'a[href="/{username}/following"] span')
    location = extract_data('span[data-testid="UserLocation"]')
    website = extract_data('a[data-testid="UserUrl"]', "href")

    return [username, bio, followers, following, location, website]

# Use a single WebDriver instance for efficiency
with webdriver.Chrome(service=service, options=chrome_options) as driver:
    for username in df["username"]:
        profile_data.append(scrape_twitter_profile(username, driver))
        time.sleep(random.uniform(5,10))  # Random delay to avoid rate limits

# Convert data to Pandas DataFrame
output_df = pd.DataFrame(profile_data, columns=["Username", "Bio", "Followers", "Following", "Location", "Website"])

# Function to insert data into MySQL
def insert_into_mysql(df):
    try:
        engine = create_engine(f"mysql+mysqlconnector://{MYSQL_DB['user']}:{MYSQL_DB['password']}@{MYSQL_DB['host']}/{MYSQL_DB['database']}")
        df.to_sql(name="twitter_profiles", con=engine, if_exists="replace", index=False)
        print("Data inserted into MySQL successfully!")
    except Exception as e:
        print(f"MySQL Error: {e}")

# Function to insert data into PostgreSQL
def insert_into_postgresql(df):
    try:
        engine = create_engine(f"postgresql+psycopg2://{POSTGRESQL_DB['user']}:{POSTGRESQL_DB['password']}@{POSTGRESQL_DB['host']}/{POSTGRESQL_DB['database']}")
        df.to_sql(name="twitter_profiles", con=engine, if_exists="replace", index=False)
        print("Data inserted into PostgreSQL successfully!")
    except Exception as e:
        print(f"PostgreSQL Error: {e}")

# Insert data into both databases
insert_into_mysql(output_df)
insert_into_postgresql(output_df)

print("Scraping complete! Data saved to MySQL and PostgreSQL databases.")


Data inserted into MySQL successfully!
Data inserted into PostgreSQL successfully!
Scraping complete! Data saved to MySQL and PostgreSQL databases.
